In [107]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.mixture import GaussianMixture
from sklearn.tree import DecisionTreeRegressor
from keras.models import Sequential
from keras.layers import Dense,BatchNormalization
from keras import optimizers,regularizers,initializers

In [80]:
def import_dataset(normalised=True,scaleMethod='Standard'):
    '''
    Imports Dataset and returns either scaled values depending upon user inputs
    
    Input:
        normalised -- boolean depending upon whether the user wants to scale the values
        scaleMethod -- Type of scaler to be used if normalised is True
    
    Output:
        (X_train,X_test,Y_train,Y_test) -- the training and testing dataset
        scaler -- used to perform inverse transform if dataset is scaled
    '''
    data = pd.read_csv('MLDataset.csv')
    dataS = data.drop('Unnamed: 0',axis=1)
    
    if normalised == False:
        scaler = 'None'
        X = dataS.iloc[:,0:3].values
        Y = dataS.iloc[:,3:].values
        X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15,random_state=0)
        
    elif scaleMethod == 'Standard':
        scaler = StandardScaler()
        scaler.fit(dataS)
        dataS = scaler.transform(dataS)
        X = dataS[:,0:3]
        Y = dataS[:,3:]
        X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15,random_state=0)
    
    elif scaleMethod == 'MinMax':
        scaler = MinMaxScaler(feature_range=(0,1))
        scaler.fit(dataS)
        dataS = scaler.transform(dataS)
        X = dataS[:,0:3]
        Y = dataS[:,3:]
        X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15,random_state=0)
    
    return X_train,X_test,Y_train,Y_test,scaler

In [3]:
def inverseTransform(scaler,*arr):
    '''
    Used to perform Inverse Transformation on normalised dataset
    
    Input:
        scaler -- Instance of Normaliser used
        *arr -- list of arrays to be concatenated
    '''
    data = np.concatenate(arr,axis=1)
    data = pd.DataFrame(data)
    arrInverse = scaler.inverse_transform(data)
    
    return arrInverse

In [4]:
def cost(y_test,y_pred):
    '''
    Calculates error of the model
    '''
    error = (y_test-y_pred)/y_test
    error = np.sum(abs(error))/(y_test.shape[0]*y_test.shape[1])*100
    
    return error

## K Neighbors Regressor 

In [85]:
Rscore = {}
for normalise in [True,False]:
    X_train,X_test,Y_train,Y_test,scaler = import_dataset(normalised=normalise)
    for nbr in range(2,100):
        for wgt in ['uniform', 'distance']:
            Model = KNeighborsRegressor(n_neighbors=nbr,weights=wgt)
            Model.fit(X_train,Y_train)
            Ytr_pred = Model.predict(X_train)
            Yts_pred = Model.predict(X_test)
            
            if normalise == False:
                error = cost(Y_test,Yts_pred)
                error_tr = cost(Y_train,Ytr_pred)
            else:
                true = inverseTransform(scaler,X_train,Y_train)
                pred = inverseTransform(scaler,X_train,Ytr_pred)
                
                error_tr = cost(true[:,3:],pred[:,3:])
                
                true = inverseTransform(scaler,X_test,Y_test)
                pred = inverseTransform(scaler,X_test,Yts_pred)
                
                error = cost(true[:,3:],pred[:,3:])
#             valTst = Model.score(X_test,Y_test)
#             val = Model.score(X_train,Y_train)
            param = {normalise,nbr,wgt}
            Rscore[(error,error_tr)]=param

In [92]:
Rscore

{(3.6773701923764315, 2.0377406344662994): {2, True, 'uniform'},
 (3.679338992146946, 0.019657641549745884): {2, True, 'distance'},
 (3.581132969907948, 2.4680730643993907): {3, True, 'uniform'},
 (3.5856075691560143, 0.018945020914940647): {3, True, 'distance'},
 (3.5308470372196337, 2.6799119232411908): {4, True, 'uniform'},
 (3.5303542706848052, 0.018791570734600898): {4, True, 'distance'},
 (3.5121478620912305, 2.818338409589552): {5, True, 'uniform'},
 (3.5095533210724277, 0.018728248976286048): {5, True, 'distance'},
 (3.4879022319495108, 2.908322518322019): {6, True, 'uniform'},
 (3.4902117049212666, 0.018676880339855085): {6, True, 'distance'},
 (3.4928574083991255, 2.975962807342564): {7, True, 'uniform'},
 (3.4898006920101046, 0.018667271959876615): {7, True, 'distance'},
 (3.496527800604853, 3.0310092507248343): {8, True, 'uniform'},
 (3.489089246191342, 0.018667271959876615): {8, True, 'distance'},
 (3.479766794201068, 3.0717959037923466): {9, True, 'uniform'},
 (3.47518896

## Decision Tree Regression

In [ ]:
Rscore = {}
for normalise in [True,False]:
    X_train,X_test,Y_train,Y_test,scaler = import_dataset(normalised=normalise,scaleMethod='MinMax')
    for ctr in ['mse','friedman_mse','mae','poisson']:
        for spt in ['best', 'random']:
            for sampsplit in range(2,10):
                for leaf in range(1,10):
                    Model = DecisionTreeRegressor(criterion=ctr,splitter=spt,min_samples_split=sampsplit,min_samples_leaf=leaf)
                    Model.fit(X_train,Y_train)
                    Ytr_pred = Model.predict(X_train)
                    Yts_pred = Model.predict(X_test)

                    if normalise == False:
                        error = cost(Y_test,Yts_pred)
                        error_tr = cost(Y_train,Ytr_pred)
                    else:
                        true = inverseTransform(scaler,X_train,Y_train)
                        pred = inverseTransform(scaler,X_train,Ytr_pred)

                        error_tr = cost(true[:,3:],pred[:,3:])

                        true = inverseTransform(scaler,X_test,Y_test)
                        pred = inverseTransform(scaler,X_test,Yts_pred)

                        error = cost(true[:,3:],pred[:,3:])
        #             valTst = Model.score(X_test,Y_test)
        #             val = Model.score(X_train,Y_train)
                    param = {normalise,ctr,spt,sampsplit,leaf}
                    Rscore[(error,error_tr)]=param

In [111]:
Rscore

{(4.18901859191914, 0.023695391517221914): {True, 'best', 'mse'},
 (4.148870100174654, 0.023695391517221914): {True, 'mse', 'random'},
 (4.187327782624826, 0.023695391517221914): {True, 'best', 'friedman_mse'},
 (4.213833546235494, 0.023695391517221914): {True, 'friedman_mse', 'random'},
 (4.139809908268621, 0.02268694296547726): {True, 'best', 'mae'},
 (4.200924136625772, 0.02268694296547726): {True, 'mae', 'random'},
 (4.150718782850905, 3.9019630539036254): {True, 'best', 'poisson'},
 (4.131593167186795, 0.22056689843233562): {True, 'poisson', 'random'},
 (4.173574392832562, 0.023695391517221914): {False, 'best', 'mse'},
 (4.137974516409422, 0.023695391517221914): {False, 'mse', 'random'},
 (4.193114987770668, 0.023695391517221914): {False, 'best', 'friedman_mse'},
 (4.13468302766094, 0.023695391517221914): {False, 'friedman_mse', 'random'},
 (4.171554864519829, 0.02268694296547726): {False, 'best', 'mae'},
 (4.287185050436317, 0.02268694296547726): {False, 'mae', 'random'},
 (4.312

## Gaussian Process Regression

## Gaussian Mixture Model

In [7]:
Rscore = {}
for normalise in [True,False]:
    X_train,X_test,Y_train,Y_test,scaler = import_dataset(normalised=normalise)
    for cpt in range(2,4):
        for cvr in ['full', 'tied','diag','spherical']:
            Model = GaussianMixture(n_components=cpt,covariance_type=cvr)
            Model.fit(X_train,Y_train[:,[0]])
            Ytr_pred = Model.predict(X_train)
            Yts_pred = Model.predict(X_test)
            Ytr_pred = Ytr_pred.reshape(Ytr_pred.shape[0],1)
            Yts_pred = Yts_pred.reshape(Yts_pred.shape[0],1)
            if normalise == False:
                error = cost(Y_test[:,[0]],Yts_pred[:,[0]])
                error_tr = cost(Y_train[:,[0]],Ytr_pred[:,[0]])
            else:
                true = inverseTransform(scaler,X_train,Y_train)
                pred = inverseTransform(scaler,X_train,Ytr_pred,Y_train[:,1:])
                
                error_tr = cost(true[:,3:],pred[:,3:])
                
                true = inverseTransform(scaler,X_test,Y_test)
                pred = inverseTransform(scaler,X_test,Yts_pred,Y_test[:,1:])
                
                error = cost(true[:,3:],pred[:,3:])
#             valTst = Model.score(X_test,Y_test)
#             val = Model.score(X_train,Y_train)
            param = {normalise,cpt,cvr}
            Rscore[(error,error_tr)]=param

In [9]:
Rscore

{(1.2660092323866023, 1.24709715142702): {2, True, 'full'},
 (1.3795775748341284, 1.3736157175060968): {2, True, 'tied'},
 (1.1258963890878102, 1.1369794626085483): {2, True, 'diag'},
 (1.3550035319133946, 1.3487678495161903): {2, True, 'spherical'},
 (1.6296353958208654, 1.6553384724433031): {3, True, 'full'},
 (1.3877418659182346, 1.4230583718113954): {3, True, 'tied'},
 (1.6117110666233059, 1.6281736313921367): {3, True, 'diag'},
 (1.7631838510860192, 1.7588711679611089): {3, True, 'spherical'},
 (99.84412447428427, 99.84298182756302): {2, False, 'full'},
 (99.93263898075874, 99.92987012490183): {2, False, 'tied'},
 (99.95150540934567, 99.95150851608659): {2, False, 'diag'},
 (99.86495075170222, 99.86282165749554): {2, False, 'spherical'},
 (99.74949729922261, 99.75188220846508): {3, False, 'full'},
 (99.81223158195307, 99.80756789724285): {3, False, 'tied'},
 (99.64331331754973, 99.64693625918024): {3, False, 'diag'},
 (99.71357609508574, 99.71552151080657): {3, False, 'spherical'}

## Artificial Neural Network

In [81]:
X_train,X_test,Y_train,Y_test,scaler = import_dataset(scaleMethod='MinMax')

In [82]:
model = Sequential()
model.add(Dense(10, input_dim=X_train.shape[1],
                kernel_initializer=initializers.RandomUniform(minval=0, maxval=1, seed=None), 
                activation='relu',
                kernel_regularizer=regularizers.l2(1e-4)))
#model.add(BatchNormalization())
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(Y_train.shape[1], activation='tanh'))
opt = optimizers.Adam(learning_rate=0.01)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=1000, batch_size=32)
met = model.evaluate(X_test, Y_test)

Epoch 1/1000
532/532 [==============================] - 1s 2ms/step - loss: 0.0818 - accuracy: 0.2618
Epoch 2/1000
532/532 [==============================] - 1s 2ms/step - loss: 0.0755 - accuracy: 0.2477
Epoch 3/1000
532/532 [==============================] - 1s 2ms/step - loss: 0.0734 - accuracy: 0.2535
Epoch 4/1000
532/532 [==============================] - 1s 2ms/step - loss: 0.0652 - accuracy: 0.2670
Epoch 5/1000
532/532 [==============================] - 1s 2ms/step - loss: 0.0617 - accuracy: 0.2666
Epoch 6/1000
532/532 [==============================] - 1s 2ms/step - loss: 0.0611 - accuracy: 0.2660
Epoch 7/1000
532/532 [==============================] - 1s 2ms/step - loss: 0.0600 - accuracy: 0.2675
Epoch 8/1000
532/532 [==============================] - 1s 2ms/step - loss: 0.0596 - accuracy: 0.2691
Epoch 9/1000
532/532 [==============================] - 1s 2ms/step - loss: 0.0589 - accuracy: 0.2669
Epoch 10/1000
532/532 [==============================] - 1s 2ms/step - loss: 0.058

532/532 [==============================] - 0s 893us/step - loss: 0.0579 - accuracy: 0.2813
Epoch 80/1000
532/532 [==============================] - 1s 965us/step - loss: 0.0578 - accuracy: 0.28680s - loss: 0.0574 - 
Epoch 81/1000
532/532 [==============================] - 1s 978us/step - loss: 0.0579 - accuracy: 0.2718
Epoch 82/1000
532/532 [==============================] - 0s 880us/step - loss: 0.0579 - accuracy: 0.2740
Epoch 83/1000
532/532 [==============================] - 0s 892us/step - loss: 0.0577 - accuracy: 0.2871
Epoch 84/1000
532/532 [==============================] - 0s 903us/step - loss: 0.0579 - accuracy: 0.2821
Epoch 85/1000
532/532 [==============================] - 0s 893us/step - loss: 0.0579 - accuracy: 0.2742
Epoch 86/1000
532/532 [==============================] - 0s 886us/step - loss: 0.0577 - accuracy: 0.2750
Epoch 87/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0579 - accuracy: 0.2753
Epoch 88/1000
532/532 [============================

532/532 [==============================] - 1s 986us/step - loss: 0.0578 - accuracy: 0.2783
Epoch 157/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0578 - accuracy: 0.2762
Epoch 158/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0577 - accuracy: 0.2721
Epoch 159/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0578 - accuracy: 0.2772
Epoch 160/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0577 - accuracy: 0.2781
Epoch 161/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0579 - accuracy: 0.2779
Epoch 162/1000
532/532 [==============================] - 1s 965us/step - loss: 0.0578 - accuracy: 0.2799
Epoch 163/1000
532/532 [==============================] - 1s 973us/step - loss: 0.0575 - accuracy: 0.2766
Epoch 164/1000
532/532 [==============================] - 1s 946us/step - loss: 0.0577 - accuracy: 0.2746
Epoch 165/1000
532/532 [==============================] - 1s 958us/step

532/532 [==============================] - 1s 994us/step - loss: 0.0578 - accuracy: 0.2720
Epoch 234/1000
532/532 [==============================] - 1s 970us/step - loss: 0.0578 - accuracy: 0.2734
Epoch 235/1000
532/532 [==============================] - 1s 984us/step - loss: 0.0579 - accuracy: 0.2738
Epoch 236/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0578 - accuracy: 0.2816
Epoch 237/1000
532/532 [==============================] - 1s 983us/step - loss: 0.0578 - accuracy: 0.2785
Epoch 238/1000
532/532 [==============================] - 1s 976us/step - loss: 0.0577 - accuracy: 0.2794
Epoch 239/1000
532/532 [==============================] - 1s 986us/step - loss: 0.0579 - accuracy: 0.2833
Epoch 240/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0576 - accuracy: 0.2779
Epoch 241/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0578 - accuracy: 0.2742
Epoch 242/1000
532/532 [==============================] - 1s 1ms/st

532/532 [==============================] - 1s 1ms/step - loss: 0.0577 - accuracy: 0.2798
Epoch 312/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0576 - accuracy: 0.2759
Epoch 313/1000
532/532 [==============================] - 1s 974us/step - loss: 0.0578 - accuracy: 0.2729
Epoch 314/1000
532/532 [==============================] - 1s 993us/step - loss: 0.0577 - accuracy: 0.2784
Epoch 315/1000
532/532 [==============================] - 1s 987us/step - loss: 0.0578 - accuracy: 0.2747
Epoch 316/1000
532/532 [==============================] - 1s 985us/step - loss: 0.0577 - accuracy: 0.2712
Epoch 317/1000
532/532 [==============================] - 1s 993us/step - loss: 0.0579 - accuracy: 0.2735
Epoch 318/1000
532/532 [==============================] - 1s 974us/step - loss: 0.0578 - accuracy: 0.2771
Epoch 319/1000
532/532 [==============================] - 1s 981us/step - loss: 0.0578 - accuracy: 0.2816
Epoch 320/1000
532/532 [==============================] - 1s 986u

...

532/532 [==============================] - 1s 1ms/step - loss: 0.0578 - accuracy: 0.2865
Epoch 622/1000
532/532 [==============================] - 1s 994us/step - loss: 0.0579 - accuracy: 0.2756
Epoch 623/1000
532/532 [==============================] - 1s 989us/step - loss: 0.0576 - accuracy: 0.2793
Epoch 624/1000
532/532 [==============================] - 1s 985us/step - loss: 0.0578 - accuracy: 0.28000s - loss: 0.057
Epoch 625/1000
532/532 [==============================] - 1s 974us/step - loss: 0.0577 - accuracy: 0.2726
Epoch 626/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0577 - accuracy: 0.2820
Epoch 627/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0577 - accuracy: 0.2722
Epoch 628/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0579 - accuracy: 0.2705
Epoch 629/1000
532/532 [==============================] - 1s 958us/step - loss: 0.0576 - accuracy: 0.2792
Epoch 630/1000
532/532 [=============================

532/532 [==============================] - 1s 1ms/step - loss: 0.0576 - accuracy: 0.2770
Epoch 699/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0579 - accuracy: 0.2736
Epoch 700/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0578 - accuracy: 0.2754
Epoch 701/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0575 - accuracy: 0.2698
Epoch 702/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0576 - accuracy: 0.2822
Epoch 703/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0575 - accuracy: 0.2837
Epoch 704/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0577 - accuracy: 0.2856
Epoch 705/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0577 - accuracy: 0.2722
Epoch 706/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0577 - accuracy: 0.2815
Epoch 707/1000
532/532 [==============================] - ETA: 0s - loss: 0.057

532/532 [==============================] - 1s 995us/step - loss: 0.0577 - accuracy: 0.2825
Epoch 776/1000
532/532 [==============================] - 1s 987us/step - loss: 0.0578 - accuracy: 0.2798
Epoch 777/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0577 - accuracy: 0.2723
Epoch 778/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0575 - accuracy: 0.2784
Epoch 779/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0576 - accuracy: 0.2796
Epoch 780/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0578 - accuracy: 0.2785
Epoch 781/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0577 - accuracy: 0.2782
Epoch 782/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0577 - accuracy: 0.2821
Epoch 783/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0580 - accuracy: 0.2768
Epoch 784/1000
532/532 [==============================] - 1s 994us/step - l

532/532 [==============================] - 1s 1ms/step - loss: 0.0576 - accuracy: 0.2781
Epoch 854/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0581 - accuracy: 0.2746
Epoch 855/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0578 - accuracy: 0.2810
Epoch 856/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0576 - accuracy: 0.2741
Epoch 857/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0577 - accuracy: 0.2805
Epoch 858/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0577 - accuracy: 0.2784
Epoch 859/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0574 - accuracy: 0.2721
Epoch 860/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0576 - accuracy: 0.2779
Epoch 861/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0578 - accuracy: 0.2855
Epoch 862/1000
532/532 [==============================] - 1s 1ms/step - loss: 0

532/532 [==============================] - 1s 1ms/step - loss: 0.0574 - accuracy: 0.2786
Epoch 932/1000
532/532 [==============================] - 1s 2ms/step - loss: 0.0578 - accuracy: 0.2771
Epoch 933/1000
532/532 [==============================] - 1s 2ms/step - loss: 0.0575 - accuracy: 0.2836
Epoch 934/1000
532/532 [==============================] - 1s 2ms/step - loss: 0.0578 - accuracy: 0.2749
Epoch 935/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0576 - accuracy: 0.2701
Epoch 936/1000
532/532 [==============================] - 1s 1ms/step - loss: 0.0577 - accuracy: 0.2780
Epoch 937/1000
532/532 [==============================] - 1s 2ms/step - loss: 0.0579 - accuracy: 0.2856
Epoch 938/1000
532/532 [==============================] - 1s 2ms/step - loss: 0.0577 - accuracy: 0.2802
Epoch 939/1000
532/532 [==============================] - 1s 2ms/step - loss: 0.0578 - accuracy: 0.2741
Epoch 940/1000
532/532 [==============================] - 1s 2ms/step - loss: 0